<img style="float: left;;" src='Figures/alinco.png' /></a>

# <center> <font color= #000047> Manipulación de Datos con la librería Pandas II


<img style="float: right; margin: 0px 0px 15px 15px;" src="https://numfocus.org/wp-content/uploads/2016/07/pandas-logo-300.png" width="400px" height="400px" />

> La clase pasada comenzamos a ver las funcionalidades de la librería [pandas](https://pandas.pydata.org/). Básicamente, aprendimos a cargar datos desde archivos `.csv`, aprendimos a seleccionar subcojuntos de estos datos mediante indización (obtener ciertas filas, o ciertas columnas, o ciertas filas y columnas determinadas). Finalmente, aprendimos a filtrar datos, es decir, a seleccionar registros de la base de datos que satisfagan cierta condición.

> Hoy nos enfocaremos a ver como reunir varias bases de datos que poseen información complementaria relativa a un mismo problema, para obtener una sola tabla con la información condensada.

Referencias:
- https://www.shanelynn.ie/merge-join-dataframes-python-pandas-index-1/
___

# 0. Motivación

En cualquier situación relacionada con ciencia de datos en la vida real, no pasarán más de 10 minutos sin que aparezca la necesidad de unir (`merge` o `join`) dos bases de datos complementarias para formar tu conjunto de datos para el análisis.

La unión de DataFrames es un proceso fundamental, que cualquier analista de datos en formación debe aprender. En esta clase aprenderemos cómo hacerlo, y en el camino responderemos las siguientes preguntas:

- ¿Qué son el `merge` o `join` de dos DataFrames?

- ¿Qué son `inner`, `outer`, `left` y `right` `joins`?



## Datos para la clase

En esta clase, seguiremos trabajando con los datos de la clase pasada:

En la carpeta "data" tenemos los archivos 

- "customers_data.csv": datos propios de los clientes de la empresa, 
- "sessions_data.csv": inicios de sesión de los clientes en la plataforma web de compras (similar a amazon),
- "transactions_data.csv": transacciones asociadas a cada inicio de sesión, y
- "products_data.csv": información de los productos. 

Podemos cargar cada uno de los archivos `.csv` como DataFrames de Pandas usando la función `read_csv()` de pandas, y examinar los contenidos de cada uno con el método `head()`.

In [1]:
# Importar pandas
import pandas as pd

In [3]:
# Cargar customers_data
customers = pd.read_csv('Data/customers_data.csv', index_col=[0])
customers.head()

,customer_id,zip_code,join_date,date_of_birth
0,1,60091,2011-04-17 10:48:33,1994-07-18
1,2,13244,2012-04-15 23:31:04,1986-08-18
2,3,13244,2011-08-13 15:42:34,2003-11-21
3,4,60091,2011-04-08 20:08:14,2006-08-15
4,5,60091,2010-07-17 05:27:50,1984-07-28


In [4]:
customers.shape

(5, 4)

In [6]:
# Cargar sessions_data
sessions = pd.read_csv('Data/sessions_data.csv', index_col=[0])
sessions.head()

,session_id,customer_id,device,session_start
0,1,2,desktop,2014-01-01 00:00:00
1,2,5,mobile,2014-01-01 00:17:20
2,3,4,mobile,2014-01-01 00:28:10
3,4,1,mobile,2014-01-01 00:44:25
4,5,4,mobile,2014-01-01 01:11:30


In [7]:
sessions.shape

(35, 4)

In [8]:
sessions.describe()

,session_id,customer_id
count,35.000000,35.000000
mean,18.000000,2.914286
std,10.246951,1.442454
min,1.000000,1.000000
25%,9.500000,2.000000
50%,18.000000,3.000000
75%,26.500000,4.000000
max,35.000000,5.000000


In [9]:
sessions.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 35 entries, 0 to 34
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   session_id     35 non-null     int64 
 1   customer_id    35 non-null     int64 
 2   device         35 non-null     object
 3   session_start  35 non-null     object
dtypes: int64(2), object(2)
memory usage: 1.4+ KB


In [10]:
# Cargar transactions_data
transactions = pd.read_csv('Data/transactions_data.csv',index_col =[0])
transactions.head()

,transaction_id,session_id,transaction_time,product_id,amount
0,298,1,2014-01-01 00:00:00,5,127.64
1,2,1,2014-01-01 00:01:05,2,109.48
2,308,1,2014-01-01 00:02:10,3,95.06
3,116,1,2014-01-01 00:03:15,4,78.92
4,371,1,2014-01-01 00:04:20,3,31.54


In [13]:
transactions.shape

(500, 5)

In [14]:
transactions.describe()

,transaction_id,session_id,product_id,amount
count,500.000000,500.000000,500.000000,500.000000
mean,250.500000,18.040000,3.036000,75.079720
std,144.481833,10.373699,1.430663,42.344853
min,1.000000,1.000000,1.000000,5.730000
25%,125.750000,8.750000,2.000000,38.122500
50%,250.500000,19.000000,3.000000,70.955000
75%,375.250000,27.000000,4.000000,110.727500
max,500.000000,35.000000,5.000000,149.950000


In [15]:
# Cargar products_data
products = pd.read_csv('Data/products_data.csv', index_col=[0])
products

,product_id,brand
0,1,B
1,2,B
2,3,B
3,4,B
4,5,A


In [16]:
products.shape

(5, 2)

## Bonus: formato de fechas y horas

Como vimos la vez pasada, es relevante en general un correcto manejo de las fechas y horas en las bases de datos.

Lo primero que se tiene que hacer para un correcto manejo de las fechas (y horas) es identificar las columnas o variables que contienen fechas. Por ejemplo, de la tabla `customers_data`:

In [17]:
customers

,customer_id,zip_code,join_date,date_of_birth
0,1,60091,2011-04-17 10:48:33,1994-07-18
1,2,13244,2012-04-15 23:31:04,1986-08-18
2,3,13244,2011-08-13 15:42:34,2003-11-21
3,4,60091,2011-04-08 20:08:14,2006-08-15
4,5,60091,2010-07-17 05:27:50,1984-07-28


Lo que sigue, es especificarle a pandas el formato en que vienen esas fechas con la función `to_datetime()` de pandas

In [18]:
# Ayuda en la función to_datetime
help(pd.to_datetime)

Help on function to_datetime in module pandas.core.tools.datetimes:

to_datetime(arg: 'DatetimeScalarOrArrayConvertible', errors: 'str' = 'raise', dayfirst: 'bool' = False, yearfirst: 'bool' = False, utc: 'bool | None' = None, format: 'str | None' = None, exact: 'bool' = True, unit: 'str | None' = None, infer_datetime_format: 'bool' = False, origin='unix', cache: 'bool' = True) -> 'DatetimeIndex | Series | DatetimeScalar | NaTType | None'
    Convert argument to datetime.
    
    This function converts a scalar, array-like, :class:`Series` or
    :class:`DataFrame`/dict-like to a pandas datetime object.
    
    Parameters
    ----------
    arg : int, float, str, datetime, list, tuple, 1-d array, Series, DataFrame/dict-like
        The object to convert to a datetime. If a :class:`DataFrame` is provided, the
        method expects minimally the following columns: :const:`"year"`,
        :const:`"month"`, :const:`"day"`.
    errors : {'ignore', 'raise', 'coerce'}, default 'raise'
   

In [19]:
# Especificar el formato de fechas en la tabla customers_data
customers.head()

,customer_id,zip_code,join_date,date_of_birth
0,1,60091,2011-04-17 10:48:33,1994-07-18
1,2,13244,2012-04-15 23:31:04,1986-08-18
2,3,13244,2011-08-13 15:42:34,2003-11-21
3,4,60091,2011-04-08 20:08:14,2006-08-15
4,5,60091,2010-07-17 05:27:50,1984-07-28


In [22]:
customers['join_date'] = pd.to_datetime(customers['join_date'],
                                      format = "%Y-%m-%d %H:%M:%S",
                                      errors = 'coerce')
customers

,customer_id,zip_code,join_date,date_of_birth
0,1,60091,2011-04-17 10:48:33,1994-07-18
1,2,13244,2012-04-15 23:31:04,1986-08-18
2,3,13244,2011-08-13 15:42:34,2003-11-21
3,4,60091,2011-04-08 20:08:14,2006-08-15
4,5,60091,2010-07-17 05:27:50,1984-07-28


In [23]:
customers['join_date']

0   2011-04-17 10:48:33
1   2012-04-15 23:31:04
2   2011-08-13 15:42:34
3   2011-04-08 20:08:14
4   2010-07-17 05:27:50
Name: join_date, dtype: datetime64[ns]

Hagan esto mismo para todas las tablas:

In [27]:
customers['date_of_birth'] = pd.to_datetime(customers['date_of_birth'],
                                          format = "%Y-%m-%d",
                                          errors = 'coerce')
customers

,customer_id,zip_code,join_date,date_of_birth
0,1,60091,2011-04-17 10:48:33,1994-07-18
1,2,13244,2012-04-15 23:31:04,1986-08-18
2,3,13244,2011-08-13 15:42:34,2003-11-21
3,4,60091,2011-04-08 20:08:14,2006-08-15
4,5,60091,2010-07-17 05:27:50,1984-07-28


In [28]:
customers['date_of_birth']

0   1994-07-18
1   1986-08-18
2   2003-11-21
3   2006-08-15
4   1984-07-28
Name: date_of_birth, dtype: datetime64[ns]

In [31]:
# Especificar el formato de fechas en la tabla sessions
sesions['session_start'].head()

0    2014-01-01 00:00:00
1    2014-01-01 00:17:20
2    2014-01-01 00:28:10
3    2014-01-01 00:44:25
4    2014-01-01 01:11:30
Name: session_start, dtype: object

In [32]:
sesions['session_start'] = pd.to_datetime(sesions['session_start'],
                                         format = "%Y-%m-%d",
                                          errors = 'coerce')


In [34]:
sesions['session_start'].head()

0   2014-01-01 00:00:00
1   2014-01-01 00:17:20
2   2014-01-01 00:28:10
3   2014-01-01 00:44:25
4   2014-01-01 01:11:30
Name: session_start, dtype: datetime64[ns]

In [35]:
transactions.head()

,transaction_id,session_id,transaction_time,product_id,amount
0,298,1,2014-01-01 00:00:00,5,127.64
1,2,1,2014-01-01 00:01:05,2,109.48
2,308,1,2014-01-01 00:02:10,3,95.06
3,116,1,2014-01-01 00:03:15,4,78.92
4,371,1,2014-01-01 00:04:20,3,31.54


In [36]:
transactions['transaction_time'] = pd.to_datetime(transactions['transaction_time'],
                                         format = "%Y-%m-%d",
                                          errors = 'coerce')

In [38]:
transactions['transaction_time'].head()

0   2014-01-01 00:00:00
1   2014-01-01 00:01:05
2   2014-01-01 00:02:10
3   2014-01-01 00:03:15
4   2014-01-01 00:04:20
Name: transaction_time, dtype: datetime64[ns]

In [ ]:
# Especificar el formato de fechas en la tabla customers


In [39]:
customers.head()

,customer_id,zip_code,join_date,date_of_birth
0,1,60091,2011-04-17 10:48:33,1994-07-18
1,2,13244,2012-04-15 23:31:04,1986-08-18
2,3,13244,2011-08-13 15:42:34,2003-11-21
3,4,60091,2011-04-08 20:08:14,2006-08-15
4,5,60091,2010-07-17 05:27:50,1984-07-28


In [40]:
sesions.head()

,session_id,customer_id,device,session_start
0,1,2,desktop,2014-01-01 00:00:00
1,2,5,mobile,2014-01-01 00:17:20
2,3,4,mobile,2014-01-01 00:28:10
3,4,1,mobile,2014-01-01 00:44:25
4,5,4,mobile,2014-01-01 01:11:30


In [41]:
products.head()

,product_id,brand
0,1,B
1,2,B
2,3,B
3,4,B
4,5,A


In [42]:
transactions.head()

,transaction_id,session_id,transaction_time,product_id,amount
0,298,1,2014-01-01 00:00:00,5,127.64
1,2,1,2014-01-01 00:01:05,2,109.48
2,308,1,2014-01-01 00:02:10,3,95.06
3,116,1,2014-01-01 00:03:15,4,78.92
4,371,1,2014-01-01 00:04:20,3,31.54


## Volviendo al problema ...

Hay columnas o variables que relacionan las tablas. Por ejemplo:

- Los clientes pueden iniciar sesión en la plataforma cuantas veces quieran. De manera que hay una relación uno a muchos entre las tablas "customers_data" y "sessions_data", mediante la variable "customer_id".

- Cuando se efectúa una transacción se supone que se está comprando un producto. Por lo tanto hay una relación uno a muchos entre las tablas "transactions_data"  y "products_data", mediante la variable "product_id".

## Problemas

Primero, quisieramos determinar cuál es el dispositivo preferido por zona.

Luego, quisiéramos determinar cuál es la marca preferida por zona.

Para resolver lo anterior necesitamos unir ("merge" o "join") nuestros datasets en uno solo para el análisis.

___
# 1. Uniendo dos DataFrames...

> “Merging” two datasets is the process of bringing two datasets together into one, and aligning the rows from each based on common attributes or columns.

Las palabras “merge” y “join” se usan indistintamente en Pandas, y en otros lenguajes como SQL y R. En Pandas, hay métodos separados “merge” y “join”, que realizan cosas similares (personalmente uso el método "merge"), y la función "merge".

Vamos a concentrarnos en el **primer problema**. En este escenario, necesitamos llevar a cabo los siguientes pasos:

- Para cada fila en el dataset `sessions_data`, debemos hacer una nueva columna que contenga el "zip_code" respectivo de cada cliente.
- Una vez hagamos esto, obtenemos la moda de los dispositivos para cada cliente.

**¿Podemos usar un ciclo?**

Claro que si. Se podría escribir un ciclo para esta tarea. Éste iría a través de cada fila en `sessions_data`, y a cada "user_id" asignar el valor de la nueva columna con la zona respectiva.

Sin embargo, usar ciclos haría nuestra tarea mucho más lenta y plagada de más código que el necesario que si se usara la función (método) `join()`.

De forma que, para estas situaciones, **nunca usar ciclos**.

Para ver cómo podemos hacer lo anterior, veamos la ayuda de la función `merge()` de pandas:

In [43]:
# Ayuda de merge()
help(pd.merge)

Help on function merge in module pandas.core.reshape.merge:

merge(left: 'DataFrame | Series', right: 'DataFrame | Series', how: 'str' = 'inner', on: 'IndexLabel | None' = None, left_on: 'IndexLabel | None' = None, right_on: 'IndexLabel | None' = None, left_index: 'bool' = False, right_index: 'bool' = False, sort: 'bool' = False, suffixes: 'Suffixes' = ('_x', '_y'), copy: 'bool' = True, indicator: 'bool' = False, validate: 'str | None' = None) -> 'DataFrame'
    Merge DataFrame or named Series objects with a database-style join.
    
    A named Series object is treated as a DataFrame with a single named column.
    
    The join is done on columns or indexes. If joining columns on
    columns, the DataFrame indexes *will be ignored*. Otherwise if joining indexes
    on indexes or indexes on a column or columns, the index will be passed on.
    When performing a cross merge, no column specifications to merge on are
    allowed.
    
    .. warning::
    
        If both key columns con

Ahora, veamos como podemos añadir el código postal a la tabla `sessions_data`, usando la función `merge()` de pandas:

In [44]:
customers.head()

,customer_id,zip_code,join_date,date_of_birth
0,1,60091,2011-04-17 10:48:33,1994-07-18
1,2,13244,2012-04-15 23:31:04,1986-08-18
2,3,13244,2011-08-13 15:42:34,2003-11-21
3,4,60091,2011-04-08 20:08:14,2006-08-15
4,5,60091,2010-07-17 05:27:50,1984-07-28


In [45]:
sesions.head()

,session_id,customer_id,device,session_start
0,1,2,desktop,2014-01-01 00:00:00
1,2,5,mobile,2014-01-01 00:17:20
2,3,4,mobile,2014-01-01 00:28:10
3,4,1,mobile,2014-01-01 00:44:25
4,5,4,mobile,2014-01-01 01:11:30


In [46]:
# Uso de la función merge()
# sessions_with_zip = pd.merge(left=sessions,
#                              right=customers[['customer_id', 'zip_code']]
#                              on="customer_id")
sessions_with_zip = pd.merge(left=sesions, right = customers[['customer_id','zip_code']], 
                             on = "customer_id")
sessions_with_zip.head()

,session_id,customer_id,device,session_start,zip_code
0,1,2,desktop,2014-01-01 00:00:00,13244
1,10,2,tablet,2014-01-01 02:31:40,13244
2,15,2,desktop,2014-01-01 03:41:00,13244
3,16,2,desktop,2014-01-01 03:49:40,13244
4,17,2,tablet,2014-01-01 04:00:30,13244


La función `merge()` es el objetivo principal de esta clase. Básicamente, la operación de unir dos DataFrames hace lo siguiente: 

- toma el DataFrame de la izquierda (argumento left=), 
- el DataFrame de la derecha (argumento right=),
- la columna donde se va a unir (argumento on=), y
- la forma en que se va a unir (argumento how=).

La variable en común sobre la que se hace la unión está especificada en el argumento `on`.

Con este resultado, podemos filtrar por zona y luego obtener la moda.

In [50]:
# Obtener la moda por zona
sessions_with_zip.loc[sessions_with_zip['zip_code']==13244, 'device'].value_counts()

desktop    7
tablet     3
mobile     3
Name: device, dtype: int64

In [52]:
#Obtener el porcentaje
100*sessions_with_zip.loc[sessions_with_zip['zip_code']==13244, 'device'].value_counts(normalize=True)

desktop    53.846154
tablet     23.076923
mobile     23.076923
Name: device, dtype: float64

In [54]:
help(pd.value_counts)

Help on function value_counts in module pandas.core.algorithms:

value_counts(values, sort: 'bool' = True, ascending: 'bool' = False, normalize: 'bool' = False, bins=None, dropna: 'bool' = True) -> 'Series'
    Compute a histogram of the counts of non-null values.
    
    Parameters
    ----------
    values : ndarray (1-d)
    sort : bool, default True
        Sort by values
    ascending : bool, default False
        Sort in ascending order
    normalize: bool, default False
        If True then compute a relative histogram
    bins : integer, optional
        Rather than count values, group them into half-open bins,
        convenience for pd.cut, only works with numeric data
    dropna : bool, default True
        Don't include counts of NaN
    
    Returns
    -------
    Series



### Usando GroupBy

In [56]:
sesion_groupby = sessions_with_zip.groupby(by='zip_code')['device'].value_counts()
sesion_groupby

zip_code  device 
13244     desktop     7
          mobile      3
          tablet      3
60091     mobile     10
          desktop     7
          tablet      5
Name: device, dtype: int64

### ¿Qué son los tipos de unión inner, left, right y outer?

En nuestro ejemplo, unimos `sessions` con `customers` sobre la columna "custumers_id". En este caso, en ambas tablas existían todos los posibles valores de "customers_id" (1, 2, 3, 4, 5).

¿Qué pasa si esta situación no se cumple?

Por ejemplo, realicemos los siguientes cambios artificiales:

In [57]:
# artificial sessions_data
art_sesions = sesions[sesions['customer_id']!=2]
art_sesions.head()

,session_id,customer_id,device,session_start
1,2,5,mobile,2014-01-01 00:17:20
2,3,4,mobile,2014-01-01 00:28:10
3,4,1,mobile,2014-01-01 00:44:25
4,5,4,mobile,2014-01-01 01:11:30
5,6,1,tablet,2014-01-01 01:23:25


In [58]:
art_sesions['customer_id'].unique()

array([5, 4, 1, 3], dtype=int64)

In [59]:
art_customers = customers[customers['customer_id']!=5]
art_customers

,customer_id,zip_code,join_date,date_of_birth
0,1,60091,2011-04-17 10:48:33,1994-07-18
1,2,13244,2012-04-15 23:31:04,1986-08-18
2,3,13244,2011-08-13 15:42:34,2003-11-21
3,4,60091,2011-04-08 20:08:14,2006-08-15


In [61]:
art_customers['customer_id'].unique()

array([1, 2, 3, 4], dtype=int64)

Por defecto, la operación `merge()` de pandas actpua con un merge tipo "inner". Un "inner merge", guarda únicamente los valores comúnes (en la columna especificada en el argumento `on=`) de ambos DataFrames.

Por ejemplo:

In [60]:
# inner join
art_sesions_merge = art_sesions.merge(right=art_customers, on = 'customer_id', how='inner')
art_sesions_merge.head()

,session_id,customer_id,device,session_start,zip_code,join_date,date_of_birth
0,3,4,mobile,2014-01-01 00:28:10,60091,2011-04-08 20:08:14,2006-08-15
1,5,4,mobile,2014-01-01 01:11:30,60091,2011-04-08 20:08:14,2006-08-15
2,8,4,tablet,2014-01-01 01:55:55,60091,2011-04-08 20:08:14,2006-08-15
3,11,4,mobile,2014-01-01 02:47:55,60091,2011-04-08 20:08:14,2006-08-15
4,12,4,desktop,2014-01-01 03:04:10,60091,2011-04-08 20:08:14,2006-08-15


In [62]:
art_sesions_merge['customer_id'].unique()

array([4, 1, 3], dtype=int64)

In [64]:
art_sesions_merge['zip_code'].unique()

array([60091, 13244], dtype=int64)

Los otros comportamientos se pueden revisar mejor a través de ejemplos:

In [65]:
# left join
art_sesions_merge_left = art_sesions.merge(right=art_customers, on ='customer_id', how = 'left')
art_sesions_merge_left.head()

,session_id,customer_id,device,session_start,zip_code,join_date,date_of_birth
0,2,5,mobile,2014-01-01 00:17:20,NaN,NaT,NaT
1,3,4,mobile,2014-01-01 00:28:10,60091.0,2011-04-08 20:08:14,2006-08-15
2,4,1,mobile,2014-01-01 00:44:25,60091.0,2011-04-17 10:48:33,1994-07-18
3,5,4,mobile,2014-01-01 01:11:30,60091.0,2011-04-08 20:08:14,2006-08-15
4,6,1,tablet,2014-01-01 01:23:25,60091.0,2011-04-17 10:48:33,1994-07-18


In [67]:
art_sesions_merge_left['customer_id'].unique()

array([5, 4, 1, 3], dtype=int64)

In [68]:
# right join
art_sesions_merge_right = art_sesions.merge(right=art_customers, on ='customer_id', how = 'right')
art_sesions_merge_right.head()

,session_id,customer_id,device,session_start,zip_code,join_date,date_of_birth
0,4.0,1,mobile,2014-01-01 00:44:25,60091,2011-04-17 10:48:33,1994-07-18
1,6.0,1,tablet,2014-01-01 01:23:25,60091,2011-04-17 10:48:33,1994-07-18
2,9.0,1,desktop,2014-01-01 02:15:25,60091,2011-04-17 10:48:33,1994-07-18
3,14.0,1,tablet,2014-01-01 03:28:00,60091,2011-04-17 10:48:33,1994-07-18
4,18.0,1,desktop,2014-01-01 04:14:35,60091,2011-04-17 10:48:33,1994-07-18


In [69]:
# outer join
art_sesions_merge_right['customer_id'].unique()

array([1, 2, 3, 4], dtype=int64)

In [70]:
art_sesions_merge_outer = art_sesions.merge(right=art_customers, on ='customer_id', how = 'outer')
art_sesions_merge_outer.head()

,session_id,customer_id,device,session_start,zip_code,join_date,date_of_birth
0,2.0,5,mobile,2014-01-01 00:17:20,NaN,NaT,NaT
1,20.0,5,desktop,2014-01-01 04:46:00,NaN,NaT,NaT
2,24.0,5,tablet,2014-01-01 05:44:30,NaN,NaT,NaT
3,28.0,5,mobile,2014-01-01 06:50:35,NaN,NaT,NaT
4,30.0,5,desktop,2014-01-01 07:27:25,NaN,NaT,NaT


In [72]:
art_sesions_merge_outer['customer_id'].unique()

array([5, 4, 1, 3, 2], dtype=int64)

In [73]:
art_sesions_merge_outer[art_sesions_merge_outer['customer_id']==2]

,session_id,customer_id,device,session_start,zip_code,join_date,date_of_birth
28,NaN,2,NaN,NaT,13244.0,2012-04-15 23:31:04,1986-08-18


Estos comportamientos son muy intuitivos a partir de sus valores.

Así mismo, sus usos son muy intuitivos (lo sabrán cuando se enfrenten a problemas reales).

### Resolvamos el problema 2

Determinar cuál es la marca preferida por zona.

In [74]:
products

,product_id,brand
0,1,B
1,2,B
2,3,B
3,4,B
4,5,A


In [76]:
customers

,customer_id,zip_code,join_date,date_of_birth
0,1,60091,2011-04-17 10:48:33,1994-07-18
1,2,13244,2012-04-15 23:31:04,1986-08-18
2,3,13244,2011-08-13 15:42:34,2003-11-21
3,4,60091,2011-04-08 20:08:14,2006-08-15
4,5,60091,2010-07-17 05:27:50,1984-07-28


In [77]:
sesions

,session_id,customer_id,device,session_start
0,1,2,desktop,2014-01-01 00:00:00
1,2,5,mobile,2014-01-01 00:17:20
2,3,4,mobile,2014-01-01 00:28:10
3,4,1,mobile,2014-01-01 00:44:25
4,5,4,mobile,2014-01-01 01:11:30
5,6,1,tablet,2014-01-01 01:23:25
6,7,3,tablet,2014-01-01 01:39:40
7,8,4,tablet,2014-01-01 01:55:55
8,9,1,desktop,2014-01-01 02:15:25
9,10,2,tablet,2014-01-01 02:31:40


In [78]:
transactions

,transaction_id,session_id,transaction_time,product_id,amount
0,298,1,2014-01-01 00:00:00,5,127.64
1,2,1,2014-01-01 00:01:05,2,109.48
2,308,1,2014-01-01 00:02:10,3,95.06
3,116,1,2014-01-01 00:03:15,4,78.92
4,371,1,2014-01-01 00:04:20,3,31.54
...,...,...,...,...,...
495,112,35,2014-01-01 08:56:15,5,55.42
496,111,35,2014-01-01 08:57:20,3,34.87
497,276,35,2014-01-01 08:58:25,1,10.94
498,266,35,2014-01-01 08:59:30,5,19.86


In [80]:
sessions_with_zip

,session_id,customer_id,device,session_start,zip_code
0,1,2,desktop,2014-01-01 00:00:00,13244
1,10,2,tablet,2014-01-01 02:31:40,13244
2,15,2,desktop,2014-01-01 03:41:00,13244
3,16,2,desktop,2014-01-01 03:49:40,13244
4,17,2,tablet,2014-01-01 04:00:30,13244
5,31,2,mobile,2014-01-01 07:42:35,13244
6,33,2,mobile,2014-01-01 08:10:45,13244
7,2,5,mobile,2014-01-01 00:17:20,60091
8,20,5,desktop,2014-01-01 04:46:00,60091
9,24,5,tablet,2014-01-01 05:44:30,60091


In [81]:
transactions_with_zip = transactions.merge(right=sessions_with_zip, on='session_id', how='inner')
transactions_with_zip.head()

,transaction_id,session_id,transaction_time,product_id,amount,customer_id,device,session_start,zip_code
0,298,1,2014-01-01 00:00:00,5,127.64,2,desktop,2014-01-01,13244
1,2,1,2014-01-01 00:01:05,2,109.48,2,desktop,2014-01-01,13244
2,308,1,2014-01-01 00:02:10,3,95.06,2,desktop,2014-01-01,13244
3,116,1,2014-01-01 00:03:15,4,78.92,2,desktop,2014-01-01,13244
4,371,1,2014-01-01 00:04:20,3,31.54,2,desktop,2014-01-01,13244


In [82]:
prod_with_zip = products.merge(right=transactions_with_zip[['product_id','zip_code']], 
                               on='product_id', 
                              how='inner')
prod_with_zip.head()

,product_id,brand,zip_code
0,1,B,13244
1,1,B,60091
2,1,B,60091
3,1,B,60091
4,1,B,60091


In [83]:
prod_with_zip.shape

(500, 3)

In [84]:
brand_groupby = prod_with_zip.groupby(by='zip_code')['brand'].value_counts()

In [85]:
brand_groupby

zip_code  brand
13244     B        149
          A         37
60091     B        247
          A         67
Name: brand, dtype: int64